# XGBoost Model Implementation with CPU Optimization

This notebook implements an XGBoost-based model for soccer match draw prediction with CPU optimization. The implementation includes:

- Model creation and configuration
- Training with early stopping
- Threshold optimization
- Hyperparameter tuning
- Model evaluation
- MLflow integration for experiment tracking

## Setup and Imports

In [32]:
# Block 1: Imports and Setup
import numpy as np
import pandas as pd
from pathlib import Path
import xgboost as xgb
import joblib
import json
import os
import sys
import time
import optuna
import mlflow
import mlflow.xgboost
import random
from typing import Any, Dict, Tuple
from datetime import datetime
from optuna.samplers import TPESampler
from optuna.pruners import MedianPruner
from sklearn.metrics import roc_auc_score
import yaml

# Add project root to Python path
project_root = str(Path().absolute().parent.parent.parent.parent)
if project_root not in sys.path:
    sys.path.append(project_root)
os.environ["PYTHONPATH"] = project_root + os.pathsep + os.environ.get("PYTHONPATH", "")
os.environ["ARROW_S3_DISABLE"] = "1"
# Configure Git executable path if available
git_executable = os.environ.get("GIT_PYTHON_GIT_EXECUTABLE")
if git_executable and os.path.exists(git_executable):
    import git
    git.refresh(git_executable)

from utils.logger import ExperimentLogger
experiment_name = "xgboost_soccer_prediction"
logger = ExperimentLogger(experiment_name)

from utils.create_evaluation_set import setup_mlflow_tracking
from models.StackedEnsemble.utils.metrics import calculate_metrics
from models.StackedEnsemble.shared.data_loader import DataLoader

# Load data
mlruns_dir = setup_mlflow_tracking(experiment_name)
dataloader = DataLoader()
X_train, y_train, X_test, y_test, X_eval, y_eval = dataloader.load_data()


2025-02-21 12:37:38,728 | INFO     | xgboost_soccer_prediction | Setting up MLflow tracking for experiment: xgboost_soccer_prediction
mlflow local_path_uri: c:/Users/szita/Documents/TheDrawCode/mlruns
2025-02-21 12:37:38,840 | INFO     | xgboost_soccer_prediction | Using existing experiment: xgboost_soccer_prediction experiment_id: 600562561289637747
2025-02-21 12:37:38,842 | INFO     | xgboost_soccer_prediction | MLflow tracking configured successfully at: c:/Users/szita/Documents/TheDrawCode/mlruns
2025-02-21 12:37:38,844 | INFO     | xgboost_soccer_prediction | Loading data splits according to ensemble strategy
2025-02-21 12:37:38,845 | INFO     | xgboost_soccer_prediction | Returning features common to all models
2025-02-21 12:37:38,847 | INFO     | xgboost_soccer_prediction | Loaded 102 selected features
2025-02-21 12:37:38,900 | INFO     | xgboost_soccer_prediction | Loaded training data from parquet: c:\Users\szita\Documents\TheDrawCode\data\api_training_final.parquet
2025-02-21

KeyboardInterrupt: 

In [17]:
# Block 2: Configuration Loading
def load_hyperparameter_space():
    """Define hyperparameter space for optimization."""
    try:
        hyperparameter_space = {
            'learning_rate': {
                'type': 'float',
                'low': 0.02,
                'high': 0.06,
                'log': True
            },
            'max_depth': {
                'type': 'int', 
                'low': 8,
                'high': 14
            },
            'min_child_weight': {
                'type': 'int',
                'low': 50,
                'high': 100
            },
            'subsample': {
                'type': 'float',
                'low': 0.4,
                'high': 0.7
            },
            'colsample_bytree': {
                'type': 'float',
                'low': 0.6,
                'high': 0.8
            },
            'reg_alpha': {
                'type': 'float',
                'low': 0.01,
                'high': 1.0,
                'log': True
            },
            'reg_lambda': {
                'type': 'float',
                'low': 7.0,
                'high': 12.0,
                'log': True
            },
            'gamma': {
                'type': 'float',
                'low': 0.1,
                'high': 3.0
            },
            'early_stopping_rounds': {
                'type': 'int',
                'low': 300,
                'high': 350
            },
            'scale_pos_weight': {
                'type': 'float',
                'low': 2.4,
                'high': 2.6
            }
            
            # 'rate_drop': {
            #     'type': 'float',
            #     'low': 0.03,
            #     'high': 0.3
            # },
            # 'skip_drop': {
            #     'type': 'float',
            #     'low': 0.2,
            #     'high': 0.8
            # }
        }
        
        # hyperparameter_space = {
        #     'learning_rate': {
        #         'type': 'float',
        #         'low': 0.04,       # Focus around ~0.046
        #         'high': 0.05,
        #         'log': True
        #     },
        #     'max_depth': {
        #         'type': 'int',
        #         'low': 10,         # Narrow around best value 11
        #         'high': 12
        #     },
        #     'min_child_weight': {
        #         'type': 'int',
        #         'low': 70,         # Focus near best 72
        #         'high': 75
        #     },
        #     'subsample': {
        #         'type': 'float',
        #         'low': 0.45,       # Focus around 0.4639
        #         'high': 0.48
        #     },
        #     'colsample_bytree': {
        #         'type': 'float',
        #         'low': 0.65,       # Focus near best 0.6671
        #         'high': 0.70
        #     },
        #     'reg_alpha': {
        #         'type': 'float',
        #         'low': 0.7,        # Around best 0.7545
        #         'high': 0.8,
        #         'log': True
        #     },
        #     'reg_lambda': {
        #         'type': 'float',
        #         'low': 9.0,        # Nearly best 9.32
        #         'high': 10.0,
        #         'log': True
        #     },
        #     'gamma': {
        #         'type': 'float',
        #         'low': 0.75,       # Around best 0.7820
        #         'high': 0.85
        #     },
        #     'early_stopping_rounds': {
        #         'type': 'int',
        #         'low': 300,        # Focused around best 327
        #         'high': 350
        #     },
        #     'scale_pos_weight': {
        #         'type': 'float',
        #         'low': 2.4,        # Around best 2.4845
        #         'high': 2.6
        #     }
        # }
        
        return hyperparameter_space
    except Exception as e:
        logger.error(f"Error creating hyperparameter space: {str(e)}")
        return None

hyperparameter_space = load_hyperparameter_space()


In [18]:
# Block 3: Model Creation
def create_model(**kwargs):
    """Create and configure XGBoost model instance."""
    try:
        params = {
            'objective': 'binary:logistic',
            'eval_metric': ['logloss', 'auc'],
            'tree_method': 'hist',
            'n_jobs': -1,
            'verbosity': 0
        }
        
        # Update with provided parameters
        params.update(kwargs)
        
        # Create model
        model = xgb.XGBClassifier(**params)
        
        return model
        
    except Exception as e:
        logger.error(f"Error creating XGBoost model: {str(e)}")
        raise


In [19]:
# Block 4: Data Conversion
def convert_to_model_format(X: pd.DataFrame, y: pd.Series = None):
    """Convert data to XGBoost DMatrix format."""
    if X is None:
        raise ValueError("The feature dataset X must not be None.")
    
    try:
        if y is not None:
            dmatrix = xgb.DMatrix(X, label=y)
        else:
            dmatrix = xgb.DMatrix(X)
        return dmatrix
    except Exception as e:
        logger.error(f"Error converting data to DMatrix: {str(e)}")
        raise


In [20]:
# Block 6: Prediction Functions
def predict(model, X, threshold=0.5):
    """Generate predictions using trained model."""
    if model is None:
        raise RuntimeError("Model must be trained before prediction")
        
    try:
        probas = model.predict_proba(X)[:, 1]
        return (probas >= threshold).astype(int)
        
    except Exception as e:
        logger.error(f"Error in model prediction: {str(e)}")
        return np.zeros(len(X))

def predict_proba(model, X):
    """Generate probability predictions."""
    if model is None:
        raise RuntimeError("Model must be trained before prediction")
        
    try:
        return model.predict_proba(X)[:, 1]
        
    except Exception as e:
        logger.error(f"Error in probability prediction: {str(e)}")
        return np.zeros(len(X))


In [21]:
def evaluate(model, X: Any, y: Any, best_threshold: float) -> Dict[str, float]:
    """Evaluate model performance on given data."""
    if model is None:
        raise RuntimeError("Model must be trained before evaluation")
    
    try:
        # Get probability predictions
        y_prob = model.predict_proba(X)[:, 1]
        
        # Get binary predictions using best threshold
        y_pred = (y_prob >= best_threshold).astype(int)
        
        # Calculate metrics
        tp = np.sum((y == 1) & (y_pred == 1))
        fp = np.sum((y == 0) & (y_pred == 1))
        fn = np.sum((y == 1) & (y_pred == 0))
        
        metrics = {
            'precision': tp / (tp + fp + 1e-10),
            'recall': tp / (tp + fn + 1e-10),
            'f1': 2 * tp / (2 * tp + fp + fn + 1e-10),
            'auc': roc_auc_score(y, y_prob),
            'brier_score': np.mean((y_prob - y) ** 2),
            'threshold': best_threshold
        }
        
        return metrics
        
    except Exception as e:
        logger.error(f"Error in model evaluation: {str(e)}")
        return {
            'precision': 0.0,
            'recall': 0.0,
            'f1': 0.0,
            'auc': 0.0,
            'brier_score': 1.0,
            'threshold': best_threshold
        }


In [22]:
def optimize_threshold(model, y_true: np.ndarray, y_prob: np.ndarray) -> float:
    """Optimize prediction threshold with focus on precision while maintaining recall above 15%."""
    try:
        best_threshold = 0.5
        best_precision = 0.0
        
        # Search through thresholds
        for threshold in np.linspace(0.3, 0.8, 51):
            y_pred = (y_prob >= threshold).astype(int)
            
            # Calculate confusion matrix components
            tp = np.sum((y_true == 1) & (y_pred == 1))
            fp = np.sum((y_true == 0) & (y_pred == 1))
            fn = np.sum((y_true == 1) & (y_pred == 0))
            
            precision = tp / (tp + fp + 1e-10)
            recall = tp / (tp + fn + 1e-10)
            
            # Only consider thresholds that maintain recall above 15%
            if recall >= 0.15:
                if precision > best_precision:
                    best_precision = precision
                    best_threshold = threshold
        
        logger.info(f"Optimized threshold: {best_threshold:.3f} with precision: {best_precision:.3f}")

        metrics = evaluate(model, X_eval, y_eval, best_threshold)
        return metrics
        
    except Exception as e:
        logger.error(f"Error optimizing threshold: {str(e)}")
        return 0.5


In [23]:
# Block 5: Training Function
def train_model(X_train, y_train, X_test, y_test, X_eval, y_eval, **kwargs):
    """Train XGBoost model with early stopping."""
    try:
        # Create model with remaining parameters
        model = create_model(**kwargs)
        
        # Create eval set for early stopping
        eval_set = [(X_test, y_test)]
        
        # Fit model with early stopping
        model.fit(
            X_train, y_train,
            eval_set=eval_set,
            verbose=False
        )
        
        # Get validation predictions
        y_prob = model.predict_proba(X_eval)[:, 1]
        metrics = optimize_threshold(model, y_eval, y_prob)
        
        return model, metrics
        
    except Exception as e:
        logger.error(f"Error training XGBoost model: {str(e)}")
        raise


In [24]:
# Block 7: Model Save/Load
def save_model(model, path, threshold=None):
    """Save XGBoost model and threshold to specified path."""
    path = Path(path)
    path.parent.mkdir(parents=True, exist_ok=True)
    
    try:
        # Save model
        joblib.dump(model, path)
        
        # Save threshold
        if threshold:
            threshold_path = path.parent / "threshold.json"
            with open(threshold_path, 'w') as f:
                json.dump({
                    'threshold': threshold,
                    'model_type': 'xgboost',
                    'params': model.get_params()
                }, f, indent=2)
                
        logger.info(f"Model saved to {path}")
        
    except Exception as e:
        logger.error(f"Error saving model: {str(e)}")
        raise

def load_model(path):
    """Load XGBoost model from specified path."""
    path = Path(path)
    if not path.exists():
        raise FileNotFoundError(f"No model file found at {path}")
        
    try:
        # Load model
        model = joblib.load(path)
        
        # Load threshold
        threshold_path = path.parent / "threshold.json"
        if threshold_path.exists():
            with open(threshold_path, 'r') as f:
                data = json.load(f)
                threshold = data.get('threshold', 0.5)
        else:
            threshold = 0.5
            
        logger.info(f"Model loaded from {path}")
        return model, threshold
        
    except Exception as e:
        logger.error(f"Error loading model: {str(e)}")
        raise


In [25]:
# Block 8: Hyperparameter Tuning
def objective(trial):
    """Optuna objective function for hyperparameter optimization."""
    try:
        params = {
            'objective': 'binary:logistic',
            'tree_method': 'hist',  # CPU-only as per requirements
            'eval_metric': ['logloss', 'auc'],
            'verbose': -1,
            'n_jobs': -1,
            # 'booster': 'hist',
            # 'normalize_type': 'tree',
            # 'sample_type': 'uniform',
            'random_state': 19
        }
        
        # Add hyperparameters from config
        hyperparameter_space = load_hyperparameter_space()
        for param_name, param_config in hyperparameter_space.items():
            if param_config['type'] == 'float':
                params[param_name] = trial.suggest_float(
                    param_name,
                    param_config['low'], 
                    param_config['high'],
                    log=param_config.get('log', False)  # Pass log if in param_config, default False
                )
            elif param_config['type'] == 'int':
                params[param_name] = trial.suggest_int(
                    param_name,
                    param_config['low'],
                    param_config['high']
                )
            elif param_config['type'] == 'categorical':
                params[param_name] = trial.suggest_categorical(
                    param_name,
                    param_config['choices']
                )
        # Train model and get metrics
        model, metrics = train_model(
            X_train, y_train,
            X_test, y_test,
            X_eval, y_eval,
            **params
        )
        
        recall = metrics.get('recall', 0.0)
        precision = metrics.get('precision', 0.0)
        
        # Report intermediate values for pruning
        trial.report(precision, step=1)
        if trial.should_prune():
            raise optuna.TrialPruned()
        
        # Optimize for precision while maintaining minimum recall
        score = precision if recall >= 0.15 else 0.0
        
        logger.info(f"Trial {trial.number}:")
        logger.info(f"  Params: {params}")
        logger.info(f"  Score: {score}")
        
        for metric_name, metric_value in metrics.items():
            trial.set_user_attr(metric_name, metric_value)
        return score

    except Exception as e:
        logger.error(f"Error in trial {trial.number}: {str(e)}")
        raise


In [26]:
# Block 9: Hypertuning Function
def hypertune_xgboost(experiment_name: str) -> float:
    """Run hyperparameter optimization with MLflow tracking."""
    try:
        # Create study
        study = optuna.create_study(
            study_name=f"xgboost_optimization_{datetime.now().strftime('%Y%m%d_%H%M')}",
            direction="maximize",
            sampler=TPESampler(seed=42),
            pruner=MedianPruner(n_startup_trials=5, n_warmup_steps=5)
        )
        
        # Start MLflow run
        with mlflow.start_run(run_name=f"xgboost_optimization_{datetime.now().strftime('%Y%m%d_%H%M')}"):
            # Log dataset info
            mlflow.log_params({
                "train_samples": len(X_train),
                "test_samples": len(X_test),
                "eval_samples": len(X_eval),
                "features": X_train.shape[1]
            })
            
            # Set tags
            mlflow.set_tags({
                "model_type": "xgboost_base",
                "optimization": "optuna",
                "cpu_only": True
            })
            
            # Optimize
            best_score = -float('inf')  # Initialize with worst possible score
            def callback(study, trial):
                nonlocal best_score
                logger.info(f"Current best score: {best_score:.4f}")
                if study.best_value > best_score:
                    best_score = study.best_value
                    logger.info(f"New best score found in trial {trial.number}: {best_score:.4f}")
                return best_score
            
            study.optimize(objective, n_trials=200, timeout=10000, callbacks=[callback])  # 2 hours timeout
            
            # Log best trial info
            logger.info(f"Best trial value: {study.best_value}")
            logger.info(f"Best parameters found: {study.best_params}")
            
            # Train final model with best parameters
            logger.info("Training final model with best parameters")
            final_model, final_metrics = train_model(
                X_train, y_train,
                X_test, y_test,
                X_eval, y_eval,
                **study.best_params
            )
            
            # Log best parameters and metrics
            mlflow.log_params(study.best_params)
            mlflow.log_metrics(final_metrics)
            
            # Create and log model signature
            input_example = pd.DataFrame(X_train[:1].copy())
            signature = mlflow.models.infer_signature(
                model_input=input_example,
                model_output=final_model.predict_proba(input_example)
            )
            
            # Log model
            mlflow.xgboost.log_model(
                xgb_model=final_model,
                artifact_path="xgboost_base_model",
                registered_model_name=f"xgboost_base_{datetime.now().strftime('%Y%m%d_%H%M')}",
                signature=signature,
                input_example=input_example
            )
            
            # Save study results
            study_path = Path(mlruns_dir) / experiment_name / "optuna_studies"
            study_path.mkdir(parents=True, exist_ok=True)
            joblib.dump(study, study_path / f"study_{datetime.now().strftime('%Y%m%d_%H%M')}.pkl")
            
            logger.info(f"Training completed with precision: {final_metrics['precision']:.4f}")
            return final_metrics['precision']
            
    except Exception as e:
        logger.error(f"Error in hyperparameter optimization: {str(e)}")
        raise


In [27]:
def train_with_precision_target(
    X_train: np.ndarray,
    y_train: np.ndarray,
    X_test: np.ndarray,
    y_test: np.ndarray,
    X_eval: np.ndarray,
    y_eval: np.ndarray,
    logger: ExperimentLogger) -> Tuple[Any, float, Dict[str, Any]]:
    """Train XGBoost model with target precision threshold."""
    
    precision = 0
    best_precision = 0
    best_recall = 0
    best_params = None
    best_seed = 0
    best_model = None
    best_threshold = 0.5
    
    # Base parameters from previous optimization
    base_params = {
        'learning_rate': 0.02054068407141532,
        'max_depth': 9,
        'min_child_weight': 54,
        'subsample': 0.5260707987602047,
        'colsample_bytree': 0.6681727685021486,
        'reg_alpha': 0.07727180013165219,
        'reg_lambda': 11.225510838289434,
        'gamma': 1.7715866142249037,
        'early_stopping_rounds': 331,
        'scale_pos_weight': 2.422289591187272,
        'tree_method': 'hist',  # Required for CPU-only training per project rules
        'objective': 'binary:logistic',
        'eval_metric': ['logloss', 'auc'],
        'verbose': -1,
        'n_jobs': -1
    }
    
    while best_precision < 0.48:  # Target precision threshold
        for random_seed in range(1, 1000):  # Try up to 1000 different seeds
            logger.info(f"Using sequential random seed: {random_seed}")
            
            # Set all random seeds
            os.environ['PYTHONHASHSEED'] = str(random_seed)
            np.random.seed(random_seed)
            random.seed(random_seed)
            base_params['random_state'] = random_seed
            
            try:
                # Create and train model
                model, metrics = train_model(
                    X_train, y_train,
                    X_test, y_test,
                    X_eval, y_eval,
                    **base_params
                )
                precision = metrics['precision']
                recall = metrics['recall']

                # Update best model if precision improved
                if precision > best_precision:
                    best_precision = precision
                    best_recall = recall
                    best_params = base_params.copy()
                    best_seed = random_seed
                    best_model = model
                    logger.info(f"New best precision: {precision:.4f} with seed {best_seed}")
                
                # Check if target precision reached
                if precision >= 0.48:
                    logger.info(f"Target precision achieved: {precision:.4f}")
                    return best_model, precision, recall, best_params
                
                logger.info(
                    f"Current precision: {precision:.4f}, "
                    f"target: 0.4800, highest precision: {best_precision:.4f}, "
                    f"best seed: {best_seed}"
                )
                
            except Exception as e:
                logger.error(f"Error training with seed {random_seed}: {str(e)}")
                continue
            
            # Clear model to free memory
            model = None
        
        # If target not reached after all seeds, return best model
        if precision < 0.48:
            logger.info(f"Target precision not reached, using best seed: {best_seed}")
            return best_model, best_precision, best_recall, best_params
            
    return best_model, best_precision, best_recall, best_params


In [28]:
def log_to_mlflow(model: object, precision: float, recall: float, params: dict, experiment_name: str) -> str:
    """Log model, metrics and parameters to MLflow.
    
    Args:
        model: Trained XGBoost model
        metrics (dict): Dictionary of metrics like precision, recall etc.
        params (dict): Model parameters used for training
        experiment_name (str): Name of the MLflow experiment
    """
    from utils.create_evaluation_set import setup_mlflow_tracking
    
    mlruns_dir = setup_mlflow_tracking(experiment_name)
    
    # Start MLflow run
    with mlflow.start_run(run_name=f"xgboost_base_{datetime.now().strftime('%Y%m%d_%H%M')}") as run:
        
        # Log parameters
        mlflow.log_params(params)
        
        # Log metrics
        mlflow.log_metrics({
            "precision": precision,
            "recall": recall
        })
        
        # Register model with timestamp
        model_name = f"xgboost_base_{datetime.now().strftime('%Y%m%d_%H%M')}"
        
        # Log model with signature
        input_example = pd.DataFrame(model.feature_names_in_[:1].copy(), dtype=float)
        signature = mlflow.models.infer_signature(
            model_input=input_example,
            model_output=predict_proba(model, input_example)
        )
        
        mlflow.xgboost.log_model(
            xgb_model=model,
            artifact_path="model",
            registered_model_name=model_name,
            signature=signature
        )
        
        # Log run ID
        run_id = run.info.run_id
        logger.info(f"Run ID: {run_id}")
        return run_id


In [29]:
def train_seed_model():
    model, precision, recall, best_params = train_with_precision_target(
                X_train, y_train,
                X_test, y_test,
                X_eval, y_eval,
                logger
            )
    print(f"Training completed with precision: {precision:.4f}")
    
    # Log to MLflow if we got a valid model
    if model is not None:
        log_to_mlflow(model, precision, recall, best_params, experiment_name)


In [30]:
if __name__ == "__main__":
    train_seed_model()
    # hypertune_xgboost(experiment_name)


[I 2025-02-21 12:21:26,452] A new study created in memory with name: xgboost_optimization_20250221_1221


2025-02-21 12:21:28,468 | INFO     | xgboost_soccer_prediction | Optimized threshold: 0.500 with precision: 0.331
2025-02-21 12:21:28,510 | INFO     | xgboost_soccer_prediction | Trial 0:
2025-02-21 12:21:28,512 | INFO     | xgboost_soccer_prediction |   Params: {'objective': 'binary:logistic', 'tree_method': 'hist', 'eval_metric': ['logloss', 'auc'], 'verbose': -1, 'n_jobs': -1, 'random_state': 19, 'learning_rate': 0.030180820828066838, 'max_depth': 14, 'min_child_weight': 87, 'subsample': 0.579597545259111, 'colsample_bytree': 0.6312037280884872, 'reg_alpha': 0.020511104188433976, 'reg_lambda': 7.2226145454655, 'gamma': 2.611910822747312, 'early_stopping_rounds': 330, 'scale_pos_weight': 2.541614515559209}
2025-02-21 12:21:28,512 | INFO     | xgboost_soccer_prediction |   Score: 0.33065442020662106


[I 2025-02-21 12:21:28,514] Trial 0 finished with value: 0.33065442020662106 and parameters: {'learning_rate': 0.030180820828066838, 'max_depth': 14, 'min_child_weight': 87, 'subsample': 0.579597545259111, 'colsample_bytree': 0.6312037280884872, 'reg_alpha': 0.020511104188433976, 'reg_lambda': 7.2226145454655, 'gamma': 2.611910822747312, 'early_stopping_rounds': 330, 'scale_pos_weight': 2.541614515559209}. Best is trial 0 with value: 0.33065442020662106.


2025-02-21 12:21:28,516 | INFO     | xgboost_soccer_prediction | Current best score: -inf
2025-02-21 12:21:28,518 | INFO     | xgboost_soccer_prediction | New best score found in trial 0: 0.3307
2025-02-21 12:21:30,134 | INFO     | xgboost_soccer_prediction | Optimized threshold: 0.530 with precision: 0.363
2025-02-21 12:21:30,177 | INFO     | xgboost_soccer_prediction | Trial 1:
2025-02-21 12:21:30,177 | INFO     | xgboost_soccer_prediction |   Params: {'objective': 'binary:logistic', 'tree_method': 'hist', 'eval_metric': ['logloss', 'auc'], 'verbose': -1, 'n_jobs': -1, 'random_state': 19, 'learning_rate': 0.020457440438566, 'max_depth': 14, 'min_child_weight': 92, 'subsample': 0.46370173320348285, 'colsample_bytree': 0.6363649934414202, 'reg_alpha': 0.023270677083837805, 'reg_lambda': 8.247380653476348, 'gamma': 1.6217936517334899, 'early_stopping_rounds': 322, 'scale_pos_weight': 2.4582458280396082}
2025-02-21 12:21:30,179 | INFO     | xgboost_soccer_prediction |   Score: 0.36292428

[I 2025-02-21 12:21:30,181] Trial 1 finished with value: 0.3629242819842395 and parameters: {'learning_rate': 0.020457440438566, 'max_depth': 14, 'min_child_weight': 92, 'subsample': 0.46370173320348285, 'colsample_bytree': 0.6363649934414202, 'reg_alpha': 0.023270677083837805, 'reg_lambda': 8.247380653476348, 'gamma': 1.6217936517334899, 'early_stopping_rounds': 322, 'scale_pos_weight': 2.4582458280396082}. Best is trial 1 with value: 0.3629242819842395.


...

[I 2025-02-21 12:27:51,590] Trial 199 finished with value: 0.34549356223168554 and parameters: {'learning_rate': 0.021999854398650338, 'max_depth': 8, 'min_child_weight': 63, 'subsample': 0.631232085928475, 'colsample_bytree': 0.6077813375844773, 'reg_alpha': 0.3255748885105559, 'reg_lambda': 11.856220675554304, 'gamma': 2.3476316607376404, 'early_stopping_rounds': 349, 'scale_pos_weight': 2.570300583876652}. Best is trial 161 with value: 0.38964577656665134.


2025-02-21 12:27:51,590 | INFO     | xgboost_soccer_prediction | Current best score: 0.3896
2025-02-21 12:27:51,592 | INFO     | xgboost_soccer_prediction | Best trial value: 0.38964577656665134
2025-02-21 12:27:51,594 | INFO     | xgboost_soccer_prediction | Best parameters found: {'learning_rate': 0.02054068407141532, 'max_depth': 9, 'min_child_weight': 54, 'subsample': 0.5260707987602047, 'colsample_bytree': 0.6681727685021486, 'reg_alpha': 0.07727180013165219, 'reg_lambda': 11.225510838289434, 'gamma': 1.7715866142249037, 'early_stopping_rounds': 331, 'scale_pos_weight': 2.422289591187272}
2025-02-21 12:27:51,594 | INFO     | xgboost_soccer_prediction | Training final model with best parameters
2025-02-21 12:27:53,496 | INFO     | xgboost_soccer_prediction | Optimized threshold: 0.520 with precision: 0.345


c:\Users\szita\.conda\envs\soccerpredictor_env\lib\site-packages\mlflow\types\utils.py:435: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
c:\Users\szita\.conda\envs\soccerpredictor_env\lib\site-packages\xgboost\core.py:158: UserWarning: [12:27:53] WARNING: D:\bld\xgboost-split_1737531313485\work\src\c_api\c_api.cc:1374: Saving model in the UBJSON format as de

2025/02/21 12:27:57 WARNING mlflow.models.model: Failed to validate serving input example {
  "dataframe_split": {
    "columns": [
      "away_yellow_cards_mean",
      "Home_offsides_mean",
      "away_attack_strength",
      "venue_encoded",
      "defensive_stability",
      "away_saves_rollingaverage",
      "venue_match_count",
      "away_corners_rollingaverage",
      "home_position_form",
      "home_corners_mean",
      "away_weighted_attack",
      "home_win_rate",
      "away_team_elo",
      "away_form_momentum_home_attack_strength_interaction",
      "away_h2h_wins",
      "fixture_id",
      "away_scoring_efficiency",
      "away_xg_form",
      "home_performance",
      "away_goal_rollingaverage",
      "elo_similarity_form_similarity",
      "away_defensive_organization",
      "home_defense_index",
      "avg_league_position",
      "away_style_compatibility",
      "date_encoded",
      "home_form_momentum_away_attack_strength_interaction",
      "home_style_compatib

2025-02-21 12:27:57,853 | INFO     | xgboost_soccer_prediction | Training completed with precision: 0.3447
